# Setup

In [14]:
from elasticsearch import (
    Elasticsearch,
    helpers
)
import pickle

In [27]:
#initialize
es = Elasticsearch()

'(lp0\r\n(dp1\r\nVposter_path\r\np2\r\nV/cezWGskPY5x7GaglTTRN4Fugfb8.jpg\r\np3\r\nsVproduction_countries\r\np4\r\n(lp5\r\n(dp6\r\nViso_3166_1\r\np7\r\nVUS\r\np8\r\nsVname\r\np9\r\nVUnited States of America\r\np10\r\nsasVrevenue\r\np11\r\nI15'


In [35]:
movies=pickle.load(open("../movies.p","r"))

In [29]:
try:
    es.indices.delete("tmdb")
except:
    pass

#create index here?
# genres.name needs to be keyword tokenized so that 'science fiction' doesn't get split on white space
# maybe create a text field with title and overview to search against
body = {
    "mappings": {
      "movie": {
        "properties": {
          "genres": {
            "properties": {
              "name": { 
                "type": "string",
                "index": "not_analyzed"}}},
          "title": {
            "type": "string",
            "analyzer": "english"}}}}}
es.indices.create("tmdb",body=body)

{u'acknowledged': True}

In [30]:
#doc indexer
def format_doc(doc):
    action = {
        "_index": "tmdb",
        "_type": "movie",
        "_id": doc['id'],
        "_source": doc
        }
    return action

def index_movies():
    actions = (format_doc(doc) for doc in movies)
    results = [details for success,details in helpers.streaming_bulk(es, actions) if not success]
    return results

In [31]:
results = index_movies()

# Examples

In [33]:
# basic facet example
body = {
    "aggs": {
        "genres": {
            "terms": {
                "field": "genres.name"
            }
        },
    }
}
es.search(index="tmdb",body=body,size=0)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'genres': {u'buckets': [{u'doc_count': 210,
     u'key': u'Action'},
    {u'doc_count': 167, u'key': u'Adventure'},
    {u'doc_count': 148, u'key': u'Thriller'},
    {u'doc_count': 117, u'key': u'Science Fiction'},
    {u'doc_count': 113, u'key': u'Drama'},
    {u'doc_count': 87, u'key': u'Fantasy'},
    {u'doc_count': 83, u'key': u'Comedy'},
    {u'doc_count': 70, u'key': u'Crime'},
    {u'doc_count': 63, u'key': u'Family'},
    {u'doc_count': 45, u'key': u'Animation'}],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 106}},
 u'hits': {u'hits': [], u'max_score': 0.0, u'total': 400},
 u'timed_out': False,
 u'took': 4}

In [34]:
# facet with a filter in place - notice the different aggregation numbers
body = {
    "fields":["title"],
    "query": {
        "bool": {
            "filter": [
              {"term": {"genres.name": "Science Fiction"}}
            ]
        }
    },
    "aggs": {
        "genres": {
            "terms": {
                "field": "genres.name"
            }
        },
    }

}
es.search(index="tmdb",body=body,size=5)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'genres': {u'buckets': [{u'doc_count': 117,
     u'key': u'Science Fiction'},
    {u'doc_count': 97, u'key': u'Action'},
    {u'doc_count': 69, u'key': u'Adventure'},
    {u'doc_count': 53, u'key': u'Thriller'},
    {u'doc_count': 15, u'key': u'Drama'},
    {u'doc_count': 14, u'key': u'Fantasy'},
    {u'doc_count': 12, u'key': u'Mystery'},
    {u'doc_count': 9, u'key': u'Comedy'},
    {u'doc_count': 7, u'key': u'Horror'},
    {u'doc_count': 6, u'key': u'Family'}],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 6}},
 u'hits': {u'hits': [{u'_id': u'49521',
    u'_index': u'tmdb',
    u'_score': 0.0,
    u'_type': u'movie',
    u'fields': {u'title': [u'Man of Steel']}},
   {u'_id': u'70981',
    u'_index': u'tmdb',
    u'_score': 0.0,
    u'_type': u'movie',
    u'fields': {u'title': [u'Prometheus']}},
   {u'_id': u'105',
    u'_index': u'tmdb',
    u'_score': 0.0,
    u'_type': u'movie',
   

# Scratch

In [37]:
# see how the genre doc conuts tum to 209 while the original language counts sum to 90 (the num docs)
body = {
    "fields":["title"],
    "query":{
        "match":{
            "genres.name": "Science Fiction"}},
    "aggs": {
        "genres": {
            "terms": {
                "field": "genres.name"
            }
        },
    }

}
es.search(index="tmdb",body=body,size=100)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'genres': {u'buckets': [{u'doc_count': 117,
     u'key': u'Science Fiction'},
    {u'doc_count': 97, u'key': u'Action'},
    {u'doc_count': 69, u'key': u'Adventure'},
    {u'doc_count': 53, u'key': u'Thriller'},
    {u'doc_count': 15, u'key': u'Drama'},
    {u'doc_count': 14, u'key': u'Fantasy'},
    {u'doc_count': 12, u'key': u'Mystery'},
    {u'doc_count': 9, u'key': u'Comedy'},
    {u'doc_count': 7, u'key': u'Horror'},
    {u'doc_count': 6, u'key': u'Family'}],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 6}},
 u'hits': {u'hits': [{u'_id': u'49521',
    u'_index': u'tmdb',
    u'_score': 2.596015,
    u'_type': u'movie',
    u'fields': {u'title': [u'Man of Steel']}},
   {u'_id': u'70981',
    u'_index': u'tmdb',
    u'_score': 2.596015,
    u'_type': u'movie',
    u'fields': {u'title': [u'Prometheus']}},
   {u'_id': u'105',
    u'_index': u'tmdb',
    u'_score': 2.596015,
    u'_type'

In [38]:
# facet with a filter in place
body = {
    "fields":["title"],
    "query": {
        "filtered": {
            "query": {
                "match":{
                    "title": "star trek"}},
            "filter": {
                "term": {
                    "genres.name": "Science Fiction"}}}},
    "aggs": {
        "genres": {
            "terms": {
                "field": "genres.name"
            }
        },
    }

}
es.search(index="tmdb",body=body,size=100)

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'aggregations': {u'genres': {u'buckets': [{u'doc_count': 8,
     u'key': u'Action'},
    {u'doc_count': 8, u'key': u'Adventure'},
    {u'doc_count': 8, u'key': u'Science Fiction'}],
   u'doc_count_error_upper_bound': 0,
   u'sum_other_doc_count': 0}},
 u'hits': {u'hits': [{u'_id': u'13475',
    u'_index': u'tmdb',
    u'_score': 3.991081,
    u'_type': u'movie',
    u'fields': {u'title': [u'Star Trek']}},
   {u'_id': u'54138',
    u'_index': u'tmdb',
    u'_score': 3.0516927,
    u'_type': u'movie',
    u'fields': {u'title': [u'Star Trek Into Darkness']}},
   {u'_id': u'1893',
    u'_index': u'tmdb',
    u'_score': 0.5672055,
    u'_type': u'movie',
    u'fields': {u'title': [u'Star Wars: Episode I - The Phantom Menace']}},
   {u'_id': u'1891',
    u'_index': u'tmdb',
    u'_score': 0.54495883,
    u'_type': u'movie',
    u'fields': {u'title': [u'Star Wars: Episode V - The Empire Strikes Back']}},
   {u'_id': u'11',
    u'_i

#Trash

In [24]:
{ "query": {
    "filtered": {
      "filter": {
        "term": {
          "genres.name": "Science Fiction"}}}},
  "aggs": {
    "genres": {
      "terms": {
        "field": "genres.name"}}}}


{'aggs': {'genres': {'terms': {'field': 'genres.name'}}},
 'query': {'filtered': {'filter': {'term': {'genres.name': 'Science Fiction'}}}}}

In [25]:
x= [
    {'doc_count': 7546, 'key': 'Drama'},
    {'doc_count': 5342, 'key': 'Comedy'},
    {'doc_count': 3878, 'key': 'Thriller'},
    {'doc_count': 3753, 'key': 'Action'},
    {'doc_count': 2623, 'key': 'Romance'},
    {'doc_count': 2165, 'key': 'Adventure'},
    {'doc_count': 1981, 'key': 'Horror'},
    {'doc_count': 1861, 'key': 'Crime'},
    {'doc_count': 1640, 'key': 'Family'},
    {'doc_count': 1597, 'key': 'Science Fiction'}]
sum([y['doc_count'] for y in x])

32386

In [26]:
es.cluster.stats()['indices'].keys()

[u'count',
 u'completion',
 u'fielddata',
 u'docs',
 u'segments',
 u'shards',
 u'query_cache',
 u'percolate',
 u'store']